   Author: Ankit Kariryaa, University of Bremen
   
   Modified by Jiawei Wei
   

#*************************************************************************************************************

Copyright (c) 2020, Ankit Kariryaa

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.s

#*************************************************************************************************************


### Overview 
The code was written by Ankit Kariryaa (Kariryaa AT uni-bremen DOT de) in 2018 (see https://doi.org/10.5281/zenodo.3978185), and some modifications were made by Jiawei Wei in 2022.

Start by labeling a part of the satellite images with the surface thermal plumes and storing the labels in shapefiles. The areas that stand for the bounding box of background areas (circles) are denoted by the 'trainingArea', core surface thermal plumes in those areas are denoted by the 'trainingPolygon', mixed surface thermal plumes (background temperature plus 1 K) in those areas are denoted by the 'trainingPlus1'. 

First, we read the trainingArea, the trainingPolygon and trainingPlus1 from three separate shape files. Then we match the trainingArea, station name and trainingPlus1 for each trainingPolygon.

Next, we read the raw satellite images (WST increment, Unit: K), and extract training image that overlap with a training area from each satellite image. The WST increment image that overlap with the training area and the corresponding label are then written to separate files.

Finally, we calculated the Euclidean distance map (Eucl_dist_map) for each WST increment image and add Eucl_dist_map as a new channel to each WST increment image.

Here, the term training area and training polygon represent all available input data, which can then be separated into training, validation, and test sets in the next notebook(s). 

### Getting started
Create a new `notebooks/config/` directory by copying the `notebooks/configTemplate/` directory and define the paths to input and output data in the `notebooks/config/Preprocessing.py` file.  
Note that in `notebooks/config/Preprocessing.py` you may need to change the default value of some variable (e.g., `raw_image_file_type` or `bands`) for your specific use case.

In [1]:
import geopandas as gps
import rasterio                  # I/O raster data (netcdf, height, geotiff, ...)
import rasterio.mask
import rasterio.warp             # Reproject raster samples
import rasterio.merge
from rasterio.transform import rowcol
import fiona                     # I/O vector data (shape, geojson, ...)
import shapely
from shapely.geometry import box, Point
from shapely.ops import unary_union
import pyproj                    # Change coordinate reference system
import pandas as pd
import json
import math
import numpy as np               # numerical array manipulation
import time
import os
from PIL import Image
import PIL.ImageDraw
from core.visualize import display_images
from core.frame_info import image_normalize

import matplotlib.pyplot as plt  # plotting tools
%matplotlib inline

from tqdm import tqdm
# from tqdm import tqdm_notebook as tqdm
import warnings                  # ignore annoying warnings
warnings.filterwarnings("ignore")

%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# Required configurations (including the input and output paths) are stored in a separate file (such as config/Preprocessing.py)
# Please provide required info in the file before continuing with this notebook. 
 
from config import Preprocessing_withLocation
# In case you are using a different folder name such as configLargeCluster, then you should import from the respective folder 
# Eg. from configLargeCluster import Preprocessing

config = Preprocessing_withLocation.Configuration()

In [3]:
#Read the training area, training polygons, plus1 
trainingArea = gps.read_file(os.path.join(config.training_base_dir, config.training_area_fn))
trainingPolygon = gps.read_file(os.path.join(config.training_base_dir, config.training_polygon_fn))
trainingPlus1 = gps.read_file(os.path.join(config.training_base_dir, config.training_plus1_fn))

print(f'Read a total of {trainingPolygon.shape[0]} object polygons and {trainingArea.shape[0]} training areas and {trainingPlus1.shape[0]} training plus1 masks')
print(f'Polygons will be assigned to training areas in the next steps.')

Read a total of 1952 object polygons and 7222 training areas and 7222 training plus1 masks
Polygons will be assigned to training areas in the next steps.


In [4]:
#Check if the training areas and the training polygons have the same crs
if trainingArea.crs  != trainingPolygon.crs:
    print('Training area CRS does not match training_polygon CRS')
    targetCRS = trainingPolygon.crs #Areas are less in number so conversion should be faster
    trainingArea = trainingArea.to_crs(targetCRS)
print(trainingPolygon.crs)
print(trainingArea.crs)
print(trainingPlus1.crs)
assert trainingPolygon.crs == trainingArea.crs

epsg:4326
epsg:4326
epsg:4326


In [5]:
for i in tqdm(trainingPolygon.index): 
    if trainingPolygon.loc[i]['geometry'].geom_type=='MultiPolygon':
        unary_union(trainingPolygon.loc[i]['geometry'])
# Assign serial order to training areas
trainingArea['order'] = range(trainingArea.shape[0])
trainingArea

# Assign serial order to training plus1 masks
trainingPlus1['order'] = range(trainingPlus1.shape[0])
trainingPlus1

# Assign serial order to training polygons
trainingPolygon['order'] = range(trainingPolygon.shape[0])
trainingPolygon

100%|███████████████████████████████████████████████████████████████████████████| 1952/1952 [00:00<00:00, 10901.85it/s]


,stationNam,system_tim,id_info,system_t_1,system_t_2,system_t_3,system_t_4,system_t_5,system_t_6,system_t_7,...,system__68,system__69,system__70,system__71,system__72,system__73,system__74,system__75,geometry,order
0,DayaBay-Lingao,1.379559e+12,LC08_121044_20130919,1.379559e+12,1.379559e+12,1.379559e+12,1.379559e+12,1.379559e+12,1.379559e+12,1.379559e+12,...,1.379559e+12,1.379559e+12,1.379559e+12,1.379559e+12,1.379559e+12,1.379559e+12,1.379559e+12,1.379559e+12,"POLYGON ((114.49884 22.54688, 114.49884 22.668...",0
1,DayaBay-Lingao,1.380941e+12,LC08_121044_20131005,1.380941e+12,1.380941e+12,1.380941e+12,1.380941e+12,1.380941e+12,1.380941e+12,1.380941e+12,...,1.380941e+12,1.380941e+12,1.380941e+12,1.380941e+12,1.380941e+12,1.380941e+12,1.380941e+12,1.380941e+12,"POLYGON ((114.49884 22.54688, 114.49884 22.668...",1
2,DayaBay-Lingao,1.412736e+12,LC08_121044_20141008,1.412736e+12,1.412736e+12,1.412736e+12,1.412736e+12,1.412736e+12,1.412736e+12,1.412736e+12,...,1.412736e+12,1.412736e+12,1.412736e+12,1.412736e+12,1.412736e+12,1.412736e+12,1.412736e+12,1.412736e+12,"POLYGON ((114.49884 22.54688, 114.49884 22.668...",2
3,DayaBay-Lingao,1.416884e+12,LC08_121044_20141125,1.416884e+12,1.416884e+12,1.416884e+12,1.416884e+12,1.416884e+12,1.416884e+12,1.416884e+12,...,1.416884e+12,1.416884e+12,1.416884e+12,1.416884e+12,1.416884e+12,1.416884e+12,1.416884e+12,1.416884e+12,"POLYGON ((114.49884 22.54688, 114.49884 22.668...",3
4,DayaBay-Lingao,1.439002e+12,LC08_121044_20150808,1.439002e+12,1.439002e+12,1.439002e+12,1.439002e+12,1.439002e+12,1.439002e+12,1.439002e+12,...,1.439002e+12,1.439002e+12,1.439002e+12,1.439002e+12,1.439002e+12,1.439002e+12,1.439002e+12,1.439002e+12,"POLYGON ((114.49884 22.54688, 114.49884 22.668...",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7217,Bruce_2,1.538238e+12,LC08_020029_20180929,1.538238e+12,1.538238e+12,1.538238e+12,1.538238e+12,1.538238e+12,1.538238e+12,1.538238e+12,...,1.538238e+12,1.538238e+12,1.538238e+12,1.538238e+12,1.538238e+12,1.538238e+12,1.538238e+12,1.538238e+12,"POLYGON ((-81.65838 44.28207, -81.65838 44.349...",7217
7218,Bruce_2,1.558974e+12,LC08_020029_20190527,1.558974e+12,1.558974e+12,1.558974e+12,1.558974e+12,1.558974e+12,1.558974e+12,1.558974e+12,...,1.558974e+12,1.558974e+12,1.558974e+12,1.558974e+12,1.558974e+12,1.558974e+12,1.558974e+12,1.558974e+12,"POLYGON ((-81.65838 44.28207, -81.65838 44.349...",7218
7219,Bruce_2,1.563121e+12,LC08_020029_20190714,1.563121e+12,1.563121e+12,1.563121e+12,1.563121e+12,1.563121e+12,1.563121e+12,1.563121e+12,...,1.563121e+12,1.563121e+12,1.563121e+12,1.563121e+12,1.563121e+12,1.563121e+12,1.563121e+12,1.563121e+12,"POLYGON ((-81.65838 44.28207, -81.65838 44.349...",7219
7220,Bruce_2,1.589387e+12,LC08_020029_20200513,1.589387e+12,1.589387e+12,1.589387e+12,1.589387e+12,1.589387e+12,1.589387e+12,1.589387e+12,...,1.589387e+12,1.589387e+12,1.589387e+12,1.589387e+12,1.589387e+12,1.589387e+12,1.589387e+12,1.589387e+12,"POLYGON ((-81.65838 44.28207, -81.65838 44.349...",7220


,allNullFla,areaFlag,stationNam,areaLimit_,core_area,type,id_info,geometry,order
0,0,1,DayaBay-Lingao,1,1.512857e+06,Polygon,LC08_121044_20130919,"POLYGON ((114.55684 22.59996, 114.55683 22.600...",0
1,0,0,DayaBay-Lingao,1,4.555738e+06,Polygon,LC08_121044_20131005,"POLYGON ((114.56597 22.59468, 114.56597 22.594...",1
2,0,0,DayaBay-Lingao,1,4.728839e+06,Polygon,LC08_121044_20141008,"POLYGON ((114.58627 22.60149, 114.58628 22.600...",2
3,0,0,DayaBay-Lingao,1,4.802820e+06,Polygon,LC08_121044_20141125,"POLYGON ((114.55912 22.60270, 114.55911 22.603...",3
4,0,0,DayaBay-Lingao,1,8.299121e+06,Polygon,LC08_121044_20150808,"POLYGON ((114.57430 22.58533, 114.57428 22.586...",4
...,...,...,...,...,...,...,...,...,...
7217,0,0,Bruce_2,1,2.683838e+06,Polygon,LC08_020029_20180929,"POLYGON ((-81.61398 44.31402, -81.61398 44.314...",7217
7218,0,1,Bruce_2,1,8.781353e+05,Polygon,LC08_020029_20190527,"POLYGON ((-81.61584 44.31158, -81.61584 44.311...",7218
7219,0,1,Bruce_2,1,1.258333e+04,Polygon,LC08_020029_20190714,"POLYGON ((-81.61211 44.31538, -81.61212 44.315...",7219
7220,0,0,Bruce_2,1,2.453743e+06,Polygon,LC08_020029_20200513,"POLYGON ((-81.61512 44.31510, -81.61512 44.315...",7220


,allNullFla,areaFlag,stationNam,areaLimit_,core_area,type,id_info,geometry,order
0,0,0,DayaBay-Lingao,1,2.327901e+06,Polygon,LC08_121044_20131005,"POLYGON ((114.56209 22.59977, 114.56208 22.600...",0
1,0,0,DayaBay-Lingao,1,2.297242e+06,Polygon,LC08_121044_20141008,"POLYGON ((114.57523 22.59915, 114.57522 22.599...",1
2,0,0,DayaBay-Lingao,1,2.119652e+06,Polygon,LC08_121044_20141125,"POLYGON ((114.56257 22.60573, 114.56254 22.607...",2
3,0,0,DayaBay-Lingao,1,5.031753e+06,Polygon,LC08_121044_20150808,"POLYGON ((114.57910 22.59434, 114.57912 22.593...",3
4,0,0,DayaBay-Lingao,1,3.765099e+06,Polygon,LC08_121044_20160607,"POLYGON ((114.60302 22.62828, 114.60303 22.627...",4
...,...,...,...,...,...,...,...,...,...
1947,0,1,Bruce_2,1,9.015040e+05,Polygon,LC08_020029_20180929,"POLYGON ((-81.61361 44.31430, -81.61361 44.314...",1947
1948,0,1,Bruce_2,1,5.338921e+05,Polygon,LC08_020029_20190527,"POLYGON ((-81.61961 44.31237, -81.61961 44.312...",1948
1949,0,1,Bruce_2,1,1.258333e+04,Polygon,LC08_020029_20190714,"POLYGON ((-81.61211 44.31538, -81.61212 44.315...",1949
1950,0,1,Bruce_2,1,1.876710e+06,Polygon,LC08_020029_20200513,"POLYGON ((-81.61286 44.31511, -81.61287 44.315...",1950


In [6]:
# read outlets location
df_info = pd.read_excel(r'I:\results\SST\landsat\location.xlsx', engine='openpyxl')
print(df_info)

              Name      Lat       Lon Location  Radius Drainage  \
0   DayaBay-Lingao  22.6076  114.5641      Bay     225  Shallow   
1        Yangjiang  21.7024  112.2713     Open      75     Deep   
2       Changjiang  19.4737  108.8834     Open     175     Deep   
3           Fuqing  25.4152  119.4365      Bay     125  Shallow   
4           Ningde  27.0524  120.2868      Bay     275  Shallow   
..             ...      ...       ...      ...     ...      ...   
69     Point_Beach  44.2822  -87.5327     Lake     175  Shallow   
70  Robert_E_Ginna  43.2805  -77.3082     Lake     125  Shallow   
71            Zion  42.4450  -87.7982     Lake     125     Deep   
72         Bruce_1  44.3448  -81.5784     Lake     275  Shallow   
73         Bruce_2  44.3158  -81.6116     Lake     125  Shallow   

          Country         Region  Capacity Start_date   End_date        CP1  \
0           China      East Asia      5802 2002-02-26        NaT 2002-12-15   
1           China      East Asia     

In [7]:
# As input we received three shapefiles, trainingArea contains the training areas/rectangles,
# trainingPolygon contains the polygon of core surface thermal plumes in those training areas,
# trainingPlus1 contains the polygon of mixed surface thermal plumes in those training areas.
# The first task is to determine the parent training area for each polygon/plus1 polygon.

def dividePolygonsInTrainingAreas(trainingPolygon, trainingArea, trainingPlus1):
    '''
    Assign annotated ploygons in to the training areas.
    '''
    # For efficiency, assigned polygons are removed from the list, we make a copy here. 
    cpTrainingPolygon = trainingPolygon.copy()
    cpTrainingPlus1 = trainingPlus1.copy()
    splitPolygons = {}
    for i in tqdm(trainingArea.index):
        spTemp = []
        allocated_j = []
        for j in cpTrainingPolygon.index:
            if (trainingArea.loc[i]['id_info']==cpTrainingPolygon.loc[j]['id_info']) and (trainingArea.loc[i]['stationNam']==cpTrainingPolygon.loc[j]['stationNam']):  
                spTemp.append(cpTrainingPolygon.loc[j])
                allocated_j.append(j)
                allocated_k = []
                for k in cpTrainingPlus1.index:
                    if (cpTrainingPolygon.loc[j]['id_info']==cpTrainingPlus1.loc[k]['id_info']) and (cpTrainingPolygon.loc[j]['stationNam']==cpTrainingPlus1.loc[k]['stationNam']):
                        splitPolygons[cpTrainingPolygon.loc[j]['order']] = {'polygons':spTemp, 'bounds':list(trainingArea.bounds.loc[i]), 'id':trainingArea.loc[i]['id_info'], 'stationName':trainingArea.loc[i]['stationNam'], 'plus1':[cpTrainingPlus1.loc[k]]} 
                        allocated_k.append(k)
                        cpTrainingPlus1 = cpTrainingPlus1.drop(allocated_k)
                        break
                cpTrainingPolygon = cpTrainingPolygon.drop(allocated_j) #narraw search range 
                break
    return splitPolygons

# areasWithPolygons contains the object polygons for each area!
areasWithPolygons = dividePolygonsInTrainingAreas(trainingPolygon, trainingArea, trainingPlus1)
# print(areasWithPolygons)
print(f'Assigned training polygons in {len(areasWithPolygons)} training areas for polygons')

100%|██████████████████████████████████████████████████████████████████████████████| 7222/7222 [50:52<00:00,  2.37it/s]

Assigned training polygons in 1952 training areas and created weighted boundaries for polygons


In [8]:
# Read the raw input images
def readInputImages(imageBaseDir, rawImageFileType, rawWSTImagePrefix):
    """
    Reads all images with prefix WST_image_prefix and image_file_type datatype in the image_base_dir directory.
    """     
    
    WSTImageFn = []
    for root, dirs, files in os.walk(imageBaseDir):
        for file in files:
#             print(os.path.join(root, file))
#             if file.endswith(rawImageFileType) and file.startswith(rawWSTImagePrefix):
            WSTImageFn.append(os.path.join(root, file))
    inputImages = list(WSTImageFn)
    return inputImages

inputImages = readInputImages(config.raw_image_base_dir, config.raw_image_file_type, config.raw_WST_image_prefix)
print(f'Found a total of {len(inputImages)} pair of raw image(s) to process!')

Found a total of 7222 pair of raw image(s) to process!


In [9]:
# For each raw satellite image, determine if it overlaps with a training area. 
# If a overlap if found, then extract + write the overlapping part of the raw image, create + write an image from training polygons.

def drawPolygons(polygons, shape, outline, fill):
    """
    From the polygons, create a numpy mask with fill value in the foreground and 0 value in the background.
    Outline (i.e the edge of the polygon) can be assigned a separate value.
    """

    mask = np.zeros(shape, dtype=np.uint8)
    mask = PIL.Image.fromarray(mask)
    draw = PIL.ImageDraw.Draw(mask)

    for polygon in polygons:
        xy = [(point[1], point[0]) for point in polygon]
        draw.polygon(xy=xy, outline=outline, fill=fill)
    mask = np.array(mask)#, dtype=bool)
    return(mask)


def rowColPolygons(areaDf, areaShape, profile, filename, outline, fill):
    """
    Convert polygons coordinates to image pixel coordinates, create annotation image using drawPolygons() and write the results into an image file.
    """
    transform = profile['transform']
    polygons = []
    for i in areaDf.index:
        gm = areaDf.loc[i]['geometry']
        a,b = zip(*list(gm.exterior.coords))
        row, col = rasterio.transform.rowcol(transform, a, b)
        zipped = list(zip(row,col)) #[list(rc) for rc in list(zip(row,col))]
        polygons.append(zipped)
    with open(filename, 'w') as outfile:  
        json.dump({'Plumes': polygons}, outfile)
    mask = drawPolygons(polygons,areaShape, outline=outline, fill=fill)
    profile['dtype'] = rasterio.int16
    profile_ann = profile.copy()
    profile_ann['count'] = 1
    with rasterio.open(filename.replace('json', 'tif'), 'w', **profile_ann) as dst:
        dst.write(mask.astype(rasterio.int16), 1)
    return(mask.astype(rasterio.int16))

def rowColPolygons_plus1(areaDf, areaShape, profile, filename, outline, fill):
    """
    Convert polygons coordinates to image pixel coordinates, create annotation image using drawPolygons() and write the results into an image file.
    """
    transform = profile['transform']
    polygons = []
    for i in areaDf.index:
        gm = areaDf.loc[i]['geometry']
        a,b = zip(*list(gm.exterior.coords))
        row, col = rasterio.transform.rowcol(transform, a, b)
        zipped = list(zip(row,col)) #[list(rc) for rc in list(zip(row,col))]
        polygons.append(zipped)
#     with open(filename, 'w') as outfile:  
#         json.dump({'Plumes': polygons}, outfile)
    mask = drawPolygons(polygons,areaShape, outline=outline, fill=fill)
    return(mask.astype(rasterio.int16))

def writeExtractedImageAndAnnotation(img, sm, Eucl_dist_map, profile, idImgStation, idImgDate, idImg, polygonsInAreaDf, plus1Mask, writePath, imagesFilename, annotationFilename, bands, writeCounter, writeCounter_hard, normalize=False):# boundariesInAreaDf, boundaryFilename, 
    """
    Write the part of raw image that overlaps with a training area into a separate image file. 
    Use rowColPolygons to create and write annotation and boundary image from polygons in the training area.
    """
    try:
        writePath_hard = r'I:\results\SST\landsat\extractWithLocation\extract50_hard_withLocation'
        if annotationFilename:
            annotation_json_filepath = os.path.join(writePath,annotationFilename+'_{}_'.format(writeCounter)+idImgStation+'_'+idImg+'.json')
            annotation_json_filepath_hard = os.path.join(writePath_hard,annotationFilename+'_{}_'.format(writeCounter_hard)+idImgStation+'_'+idImg+'.json')
            # The object is given a value of 1, the outline or the border of the object is given a value of 0 and rest of the image/background is given a a value of 0
#             mask = rowColPolygons(polygonsInAreaDf,(sm[0].shape[1], sm[0].shape[2]), profile, annotation_json_filepath, outline=1, fill = 1) 
            mask_plus1 = rowColPolygons_plus1(plus1Mask,(sm[0].shape[1], sm[0].shape[2]), profile, annotation_json_filepath, outline=1, fill = 1) 
        for band, imFn in zip(bands, imagesFilename):
            # Rasterio reads file channel first, so the sm[0] has the shape [1 or ch_count, x,y]
            # If raster has multiple channels, then bands will be [0, 1, ...] otherwise simply [0]
            profile['dtype'] = rasterio.float32
            dt = sm[0][band].astype(profile['dtype']) 
            dt_ls = [dt, Eucl_dist_map]
            profile['count'] = 2
            
            #mask outside of plus1 
            dt[mask_plus1==0] = np.nan
            Eucl_dist_map[mask_plus1==0] = np.nan
                        
            #mask abnormal value (manual inspection)
            if idImgStation == 'Takahama':
                dt[dt>=10] = np.nan
            if idImgStation == 'Ohi':
                dt[dt>=10] = np.nan
       
            
            if normalize: # Note: If the raster contains None values, then you should normalize it separately by calculating the mean and std without those values.
                dt = image_normalize(dt, axis=None) #  Normalize the image along the width and height, and since here we only have one channel we pass axis as None          
            with rasterio.open(os.path.join(writePath, imFn+'_{}_'.format(writeCounter)+idImgStation+'_'+idImg+'.tif'), 'w', **profile) as dst: 
                    for band_id, src in enumerate(dt_ls, start=1):
                        dst.write(src, band_id)
            # mask = rowColPolygons(polygonsInAreaDf,(sm[0].shape[1], sm[0].shape[2]), profile, annotation_json_filepath, outline=1, fill = 1) 
        return(writeCounter+1, writeCounter_hard)
    except Exception as e:
        print(e)
        print("Something nasty happened, could not write the annotation or the mask file!")
        return writeCounter, writeCounter_hard
        
        
def findOverlap(img, areasWithPolygons, writePath, imageFilename, annotationFilename, bands, writeCounter=1, writeCounter_hard=1):
    """
    Finds overlap of image with a training area.
    Use writeExtractedImageAndAnnotation() to write the overlapping training area and corresponding polygons in separate image files.
    """
    overlapppedAreas = set()
    for areaID, areaInfo in areasWithPolygons.items():                                 
        #Convert the polygons in the area in a dataframe and get the bounds of the area. 
        polygonsInAreaDf = gps.GeoDataFrame(areaInfo['polygons'])
        plus1Mask = gps.GeoDataFrame(areaInfo['plus1'])
        idArea_sub = areaInfo['id'].find('L')
        idArea = areaInfo['id'][idArea_sub:]
        idAreaStation = areaInfo['stationName']
        
        idImg_sub = img.name.rfind('L')
        idImg = img.name[idImg_sub:-4]
        idImgDate = img.name[idImg_sub+12:-4]
        idImgStation_sub = img.name.find('_')
        idImgStation = img.name[idImgStation_sub+1:idImg_sub-1]
        
        #test whether image is included in df_info
        t = df_info.loc[df_info['Name']==idImgStation, ['Lon']]
        if t.empty:
            print('Empty!'+idImgStation)
            break
        
        bboxArea = box(*areaInfo['bounds'])
        bboxImg = box(*img.bounds)
        #Extract the window if area is in the image
        if idArea==idImg and idAreaStation==idImgStation: 
            profile = img.profile  
            sm = rasterio.mask.mask(img, [bboxArea], all_touched=True, crop=True )
            profile['height'] = sm[0].shape[1]
            profile['width'] = sm[0].shape[2]
            profile['transform'] = sm[1]   
            # That's a problem with rasterio, if the height and the width are less then 256 it throws: ValueError: blockysize exceeds raster height 
            # So I set the blockxsize and blockysize to prevent this problem
            profile['blockxsize'] = 32 
            profile['blockysize'] = 32
            profile['count'] = 1
            profile['dtype'] = rasterio.float32
            
            #obtain the lat lon for each outlet (ol), calcualate the Euclidean distance map
            lat = df_info.loc[df_info['Name']==idImgStation, ['Lat']].iloc[0, 0]
            lon = df_info.loc[df_info['Name']==idImgStation, ['Lon']].iloc[0, 0]
            transform = profile['transform']
            row_ol, col_ol = rasterio.transform.rowcol(transform, lon, lat)
            row_num = profile['height'] 
            col_num = profile['width']
            row_ol_map = np.ones((row_num, col_num))*row_ol
            col_ol_map = np.ones((row_num, col_num))*col_ol

            row_map = np.tile(np.arange(0, row_num).reshape(row_num, 1), (1, col_num))
            col_map = np.tile(np.arange(0, col_num), (row_num, 1))
            Eucl_dist_map = np.power((np.power((row_map-row_ol_map), 2) + np.power((col_map-col_ol_map), 2)), 0.5).astype(profile['dtype'])

            
            # writeExtractedImageAndAnnotation writes the image, annotation and boundaries and returns the counter of the next file to write. 
            writeCounter, writeCounter_hard = writeExtractedImageAndAnnotation(img, sm, Eucl_dist_map, profile, idImgStation, idImgDate, idImg, polygonsInAreaDf, plus1Mask, writePath, imageFilename, annotationFilename, bands, writeCounter, writeCounter_hard)#boundariesInAreaDf, boundaryFilename, 
            print(idImgStation+'_'+idImg)
            overlapppedAreas.add(areaID)
            areasWithPolygons.pop(areaID)
            break
    return(writeCounter, writeCounter_hard, overlapppedAreas)


def extractAreasThatOverlapWithTrainingData(inputImages, areasWithPolygons, writePath, WSTFilename, annotationFilename, bands, writeCounter, writeCounter_hard):
    """
    Iterates over raw WST image and using findOverlap() extract areas that overlap with training data. The overlapping areas in raw images are written in a separate file, and annotation and boundary file are created from polygons in the overlapping areas.
    """
    if not os.path.exists(writePath):
        os.makedirs(writePath)
        
    overlapppedAreas = set()
    cpAreasWithPolygons = areasWithPolygons.copy()
    for imgs in tqdm(inputImages):
        WSTImg = rasterio.open(imgs, 'r+')

        ncWST, ncWST_hard, imOverlapppedAreasWST = findOverlap(WSTImg, cpAreasWithPolygons, writePath=writePath, imageFilename=[WSTFilename], annotationFilename=annotationFilename, bands=bands, writeCounter=writeCounter, writeCounter_hard=writeCounter_hard)#boundaryFilename=boundaryFilename, 
        
        writeCounter = ncWST
        writeCounter_hard = ncWST_hard

        if overlapppedAreas.intersection(imOverlapppedAreasWST):
            print(f'Information: Training area(s) {overlapppedAreas.intersection(imOverlapppedAreasWST)} spans over multiple raw images. This is common and expected in many cases. A part was found to overlap with current input image.')
        overlapppedAreas.update(imOverlapppedAreasWST)
    
    allAreas = set(areasWithPolygons.keys())
    if allAreas.difference(overlapppedAreas):
        print(f'Warning: Could not find a raw image correspoinding to {allAreas.difference(overlapppedAreas)} areas. Make sure that you have provided the correct paths!')
    

In [ ]:
# Run the main function for extracting part of WST images that overlap with training areas
writeCounter=481
writeCounter_hard=0
extractAreasThatOverlapWithTrainingData(inputImages, areasWithPolygons, config.path_to_write, config.extracted_WST_filename, config.extracted_annotation_filename, config.bands, writeCounter, writeCounter_hard)

In [ ]:
# Display extracted image
sampleImage = '_1.png'
fn = os.path.join(config.path_to_write, config.extracted_WST_filename + sampleImage )

WST_img = Image.open(fn)
read_WST_img = np.array(WST_img)
annotation_im = Image.open(fn.replace(config.extracted_WST_filename ,config.extracted_annotation_filename))
read_annotation = np.array(annotation_im)
all_images = np.array([read_WST_img, read_annotation])
display_images(np.expand_dims(np.transpose(all_images, axes=(1,2,0)), axis=0))